In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from keras.layers import Input, Dense, Lambda, Add, Reshape
from keras.models import Model
from keras import backend as K
from keras import metrics
from keras.datasets import mnist
import tensorflow as tf
from keras import objectives

Using TensorFlow backend.


In [2]:
batch_size = 100
original_dim = 784
latent_dim = 2
intermediate_dim = 256
epochs = 50
epsilon_std = 1.0

## Encoder

In [3]:
x = Input(batch_shape=(batch_size, original_dim))
h = Dense(intermediate_dim, activation='relu')(x)
z_mean_0 = Dense(latent_dim)(h)
z_std_0 = Dense(latent_dim, activation="softplus")(h)
encoder = Model(x, [z_mean_0,z_std_0])

## Latent models

In [4]:
mask = K.variable(np.fromfunction(lambda x,y: x>y, (latent_dim, latent_dim)).astype(np.int32))

In [5]:
def sample_eps(batch_size, latent_dim, epsilon_std):
    """Create a function to sample N(0, epsilon_std) vectors"""
    return lambda args: K.random_normal(shape=(batch_size, latent_dim),
                                        mean=0.,
                                        stddev=epsilon_std)
def sample_z0(args):
    """Sample from N(mu, sigma) where sigma is the stddev !!!"""
    z_mean, z_std, epsilon = args
    # generate z0 according to N(z_mean, z_std)
    z0 = z_mean + K.exp(K.log(z_std + 1e-8)) epsilon
    return z0

## Decoder

In [6]:
eps = Lambda(sample_eps(batch_size, latent_dim, epsilon_std), name='sample_eps')([z_mean_0, z_std_0])
z0 = Lambda(sample_z0, name='sample_z0')([z_mean_0, z_std_0, eps])

# we instantiate these layers separately so as to reuse them later
decoder_h = Dense(intermediate_dim, activation='relu')
decoder_mean = Dense(original_dim, activation='sigmoid')
h_decoded = decoder_h(z0)
x_decoded_mean = decoder_mean(h_decoded)

## VAE

In [7]:
def log_normal(e, log_var):
    c = math.log(2*math.pi)
    result = -0.5 * K.sum(c + log_var + K.square(e), -1)
    return result

In [8]:
n_sample = 20
beta = 0
def vae_loss(x, x_decoded_mean):
    xent_loss = original_dim * metrics.binary_crossentropy(x, x_decoded_mean)
    # kl divergence
    # sampling for estimating the expectations
    for k in range(n_sample):
        epsilon = K.random_normal(shape=(batch_size, latent_dim, 1), mean=0.,
                              stddev=1.0)  # used for every z_i sampling
        z0_ = z_mean_0 + z_std_0 * tf.squeeze(epsilon) + tf.squeeze(tf.matmul(mask*L_0, epsilon))

        try:
            loss += log_normal(tf.squeeze(epsilon), 2*K.log(z_std_0+1e-8))
        except NameError:
            loss = log_normal(tf.squeeze(epsilon), 2*K.log(z_std_0+1e-8))

        loss -= K.sum(log_stdnormal(z0_) , -1)
    kl_loss = loss / n_sample
    
    return xent_loss + kl_loss

In [9]:
vae = Model(x, x_decoded_mean)
vae.compile(optimizer='rmsprop', loss=vae_loss)

In [10]:
# train the VAE on MNIST digits
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

In [11]:
x_train.shape

(60000, 784)

In [12]:
from keras.callbacks import TensorBoard

tb = TensorBoard(log_dir='./log/MNIST/full_Gaussian_posterior')

In [13]:
vae.fit(x_train, x_train,
        shuffle=True,
        verbose=0,
        callbacks=[tb],
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(x_test, x_test))

## Play with the trained model

In [14]:
# build a model to project inputs on the latent space
encoder = Model(x, z_mean_0)

NameError: name 'z_mean' is not defined

In [ ]:
# display a 2D plot of the digit classes in the latent space
x_test_encoded = encoder.predict(x_test, batch_size=batch_size)
plt.figure(figsize=(6, 6))
plt.scatter(x_test_encoded[:, 0], x_test_encoded[:, 1], c=y_test)
plt.colorbar()
plt.show()

# build a digit generator that can sample from the learned distribution
decoder_input = Input(shape=(latent_dim,))
_h_decoded = decoder_h(decoder_input)
_x_decoded_mean = decoder_mean(_h_decoded)
generator = Model(decoder_input, _x_decoded_mean)

# display a 2D manifold of the digits
n = 15  # figure with 15x15 digits
digit_size = 28
figure = np.zeros((digit_size * n, digit_size * n))
# linearly spaced coordinates on the unit square were transformed through the inverse CDF (ppf) of the Gaussian
# to produce values of the latent variables z, since the prior of the latent space is Gaussian
grid_x = norm.ppf(np.linspace(0.05, 0.95, n))
grid_y = norm.ppf(np.linspace(0.05, 0.95, n))

for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
        z_sample = np.array([[xi, yi]])
        x_decoded = generator.predict(z_sample)
        digit = x_decoded[0].reshape(digit_size, digit_size)
        figure[i * digit_size: (i + 1) * digit_size,
               j * digit_size: (j + 1) * digit_size] = digit

plt.figure(figsize=(10, 10))
plt.imshow(figure, cmap='Greys_r')
plt.show()